In [6]:

import pytorch_lightning
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureType,
)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch, DataLoader
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import random
import nibabel as nib
import numpy as np
import SimpleITK as sitk

In [7]:
image = nib.load('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_image.nii.gz')
image_np = np.array(image.dataobj)
print(image_np.shape)

label = nib.load('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_label.nii.gz')
label_np = np.array(label.dataobj)
print(label_np.shape)

(512, 512, 96)
(512, 512, 96)


In [12]:

images = []
labels = []
for root, dirs, files in os.walk('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/'):
    for file in files:
        if file.endswith("_image.nii.gz"):
            images.append(os.path.join(root, file))
        if file.endswith("_label.nii.gz"):
            labels.append(os.path.join(root, file))
images = sorted(images)
labels = sorted(labels)

data = list(zip(images, labels))

resample = sitk.ResampleImageFilter()
resample.SetSize((96, 512, 512))

for i in range(len(data)):
    print(i)
    im = nib.load(data[i][0])
    im_np = np.array(im.dataobj)
    print(im_np.shape)
    #im_sitk = sitk.GetImageFromArray(im_np)
    im_sitk = sitk.ReadImage(data[i][0], imageIO="NiftiImageIO")
    im_resampled = resample.Execute(im_sitk)
    im_re_np = sitk.GetArrayFromImage(im_resampled)
    print(im_re_np.shape)
    la = nib.load(data[i][1])
    la_np = np.array(la.dataobj)
    print(la_np.shape)

0
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
1
(290, 320, 64)
(512, 512, 96)
(290, 320, 64)
2
(512, 512, 84)
(512, 512, 96)
(512, 512, 84)
3
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
4
(290, 320, 72)
(512, 512, 96)
(290, 320, 72)
5
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
6
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
7
(512, 512, 88)
(512, 512, 96)
(512, 512, 88)
8
(290, 320, 80)
(512, 512, 96)
(290, 320, 80)
9
(290, 320, 72)
(512, 512, 96)
(290, 320, 72)
10
(290, 320, 88)
(512, 512, 96)
(290, 320, 88)
11
(290, 320, 72)
(512, 512, 96)
(290, 320, 72)
12
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
13
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
14
(290, 320, 72)
(512, 512, 96)
(290, 320, 72)
15
(290, 320, 56)
(512, 512, 96)
(290, 320, 56)
16
(512, 512, 92)
(512, 512, 96)
(512, 512, 92)
17
(512, 512, 96)
(512, 512, 96)
(512, 512, 96)
18
(512, 512, 88)
(512, 512, 96)
(512, 512, 88)
19
(512, 512, 84)
(512, 512, 96)
(512, 512, 84)
20
(512, 512, 92)
(512, 512, 96)
(512, 512, 92)
21